In [1]:
import torch
import torch.nn.functional as F
import matplotlib.pyplot as plt
import os
%matplotlib inline

In [3]:
d = os.getcwd()
os.chdir("..")
os.getcwd()
words = open(os.getcwd()+'\\names.txt','r').read().splitlines()
words[:1]

['emma']

In [4]:
len(words)

32033

In [6]:
stoi.items()

dict_items([('a', 1), ('b', 2), ('c', 3), ('d', 4), ('e', 5), ('f', 6), ('g', 7), ('h', 8), ('i', 9), ('j', 10), ('k', 11), ('l', 12), ('m', 13), ('n', 14), ('o', 15), ('p', 16), ('q', 17), ('r', 18), ('s', 19), ('t', 20), ('u', 21), ('v', 22), ('w', 23), ('x', 24), ('y', 25), ('z', 26), ('.', 0)])

In [5]:
#vocabulary and mappings to integers
chars = sorted(list(set(''.join(words))))
stoi = {s:i+1 for i,s in enumerate(chars)}
stoi['.'] = 0
itos = {i:s for s,i in stoi.items()}

In [64]:
block_size = 3
X,Y = [],[]
for w in words:
    context = [0]*block_size
    for ch in w+'.':
        idx = stoi[ch]
        Y.append(idx)
        X.append(context)
        #print(''.join(itos[i] for i in context),'---->',itos[idx])
        context = context[1:]+[idx]
X = torch.tensor(X)
Y = torch.tensor(Y)

In [65]:
X.shape,X.dtype,Y.shape,Y.dtype

(torch.Size([228146, 3]), torch.int64, torch.Size([228146]), torch.int64)

In [66]:
#Embedding the words into a two-dimensional space
C = torch.randn((27,2))

![wtf](mlp.png)

In [30]:
emb = C[X]
emb.shape

torch.Size([32, 3, 2])

In [67]:
#First dimension of W1 is 6 because context length is 3 and dimension space is 2 ->
#each input is 6 integers long
#100 neurons
W1 = torch.randn((block_size*2,100))
b1 = torch.randn(100)

In [68]:
#-1 means pytorch predicts what that value will have to be because number of elements have to be the same
h = torch.tanh(emb.view(-1,6)@W1+b1)

In [37]:
h.shape

torch.Size([32, 100])

In [69]:
W2 = torch.randn((100,27))
b2 = torch.randn(27)

In [40]:
logits = h@W2 + b2

In [41]:
logits.shape

torch.Size([32, 27])

In [53]:
#softmax approach
counts = logits.exp()
prob = counts / counts.sum(1,keepdim = True)
print(prob.shape)
print(prob[0],prob[0].sum())
#this gives the probability of picking the correct character for each test case
#prob[0][Y[0]],prob[1][Y[1]],...
prob[torch.arange(32),Y]
loss = -prob[torch.arange(32),Y].log().mean()
loss

torch.Size([32, 27])
tensor([3.1414e-09, 7.6173e-02, 1.5729e-03, 5.7190e-10, 6.2735e-06, 2.1362e-06,
        2.0557e-06, 3.8085e-01, 3.9338e-10, 8.4278e-08, 9.7675e-08, 3.7589e-04,
        5.6260e-05, 1.0982e-06, 4.9919e-04, 1.3832e-07, 2.9817e-05, 3.9778e-09,
        5.6752e-05, 7.3410e-06, 6.8230e-03, 6.8950e-07, 4.5052e-06, 8.4447e-07,
        5.3102e-01, 2.5087e-03, 3.8081e-06]) tensor(1.)


tensor(16.8916)

In [55]:
#better approach, cross entropy
loss = F.cross_entropy(logits,Y)
loss

tensor(16.8916)

----------------------------------

In [94]:
C = torch.randn((27,2))
W1 = torch.randn((6,100))
b1 = torch.randn(100)
W2 = torch.randn((100,27))
b2 = torch.randn(27)

In [95]:
parameters = [C,W1,b1,W2,b2]
sum(p.nelement() for p in parameters)

3481

In [96]:
for p in parameters:
   p.requires_grad = True

In [97]:
lre = torch.linspace(-3,0,1000)
lrs = 10**lre

In [103]:
lri = []
lossi = []
for i in range(10000):
    #creating minibatch
    ix = torch.randint(0,X.shape[0],(32,))

    #forward pass
    emb = C[X[ix]]
    h = torch.tanh(emb.view(-1,6)@W1+b1)
    logits = h@W2 + b2
    loss = F.cross_entropy(logits,Y[ix])
    #print(loss.item())
    #backward pass
    for p in parameters:
        p.grad = None
    loss.backward()
    #update
    #lr = lrs[i]
    for p in parameters:
        p.data += -0.01*p.grad

    #lri.append(lre[i])
    #lossi.append(loss.item())  
print(loss.item())

2.0378358364105225


In [104]:
emb = C[X]
h = torch.tanh(emb.view(-1,6)@W1+b1)
logits = h@W2 + b2
loss = F.cross_entropy(logits,Y)
loss

tensor(2.2451, grad_fn=<NllLossBackward0>)